# Kaggle Challenge Day 1: Permutation importance
Machine Learning for Insights challenge

- Feature engineering - involves repeatedly creating new features using transformations of your raw data or features you've previously created

- Which features have the biggest impact on predictions? This concept is called feature importance

**Linear Algebra Topics such as:
- Principal Component Analysis (PCA)
- Singular Value Decomposition (SVD)
- Eigendecomposition of a matrix
- LU Decomposition
- QR Decomposition/Factorization
- Symmetric Matrices
- Orthogonalization & Orthonormalization
- Matrix Operations, Projections, Eigenvalues & Eigenvectors, Vector Spaces and Norms are needed for understanding the optimization methods used for machine learning**



#Code example

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('../Data Science/Kaggle_Challenges/FIFA 2018 Statistics.csv')
y = (data['Man of the Match'] == "Yes")  # Convert from string "Yes"/"No" to binary
feature_names = [i for i in data.columns if data[i].dtype in [np.int64]]
X = data[feature_names]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
my_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

# Permutation Importance
Taxi Fare Prediction competition

# Loading data, dividing, modeling and EDA below
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

data = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv', nrows=50000)

# Remove data with extreme outlier coordinates or negative fares
data = data.query('pickup_latitude > 40.7 and pickup_latitude < 40.8 and ' +
                  'dropoff_latitude > 40.7 and dropoff_latitude < 40.8 and ' +
                  'pickup_longitude > -74 and pickup_longitude < -73.9 and ' +
                  'dropoff_longitude > -74 and dropoff_longitude < -73.9 and ' +
                  'fare_amount > 0'
                  )

y = data.fare_amount

# Features


train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# number of trees in forest
# random_state =1 seed
# .fit(fits the data)

first_model = RandomForestRegressor(n_estimators=30, random_state=1).fit(train_X, train_y)


# understanding training data
train_X.describe()
train_y.describe()



In [ ]:
# import eli5
# from eli5.sklearn import PermutationImportance

# Make a small change to the code below to use in this problem. 
perm = PermutationImportance(first_model, random_state=1).fit(val_X, val_y)

# uncomment the following line to check your results
 q_2.check()

# uncomment the following line to visualize your results
# eli5.show_weights(perm, feature_names = val_X.columns.tolist())

eli5.show_weights